## Method 3: Append, instead of Using pd.concat

All code below is referenced from Lecture_1_2.ipynb provided by Gittu George for DSCI 525

In [1]:
# import packages

import dask.dataframe as dd
import re
import os
import glob
import sys
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from memory_profiler import memory_usage
from os import listdir
from functools import reduce

In [2]:
%load_ext memory_profiler

In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want

Next, we download the data:

In [5]:
# make directory if missing
os.makedirs(output_directory, exist_ok=True)

# download missing files
files_to_dl = ["data.zip"]
for item in filter(lambda x: x['name'] in files_to_dl, files):
    filename = os.path.join(output_directory, item["name"])
    if not os.path.isfile(filename):
        urlretrieve(item["download_url"], filename)

In [6]:
# list all CSVs
csvs = glob.glob(output_directory + '*.csv')

# As per Tom's guidance, we can exclude the annoying CSV that is formatted differently
csvs = [x for x in csvs if "observed" not in x]

In [7]:
%%time
%%memit

dat = pd.DataFrame()
for file in csvs:
    tempdf = pd.read_csv(file)
    tempdf['model'] = file.split('_daily')[0]
    dat = dat.append(tempdf)

peak memory: 12750.94 MiB, increment: 12652.15 MiB
CPU times: user 2min 12s, sys: 37 s, total: 2min 49s
Wall time: 2min 54s


In [8]:
assert dat.shape[0] == 62467843